In [126]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder

In [144]:
! pwd

/Users/alphonsowoodbury/DS/Projects/aw_flatiron_group_projects/Module3_Project


In [146]:
labels = pd.read_csv('data/labels.csv')
labels

,id,status_group
0,69572,functional
1,8776,functional
2,34310,functional
3,67743,non functional
4,19728,functional
...,...,...
59395,60739,functional
59396,27263,functional
59397,37057,functional
59398,31282,functional


In [127]:
df = pd.read_csv('cleaned_water.csv',index_col=0)
df = df.drop(columns=['latitude','longitude','id','amount_tsh','gps_height','population'])
df.head()

,status_group,funder,installer,basin,region,lga,ward,public_meeting,scheme_management,permit,extraction_type_class,management,payment_type,water_quality,quantity,source,source_class,waterpoint_type,age
0,functional,Roman,Roman,Lake Nyasa,Iringa,Ludewa,Mundindi,True,VWC,False,gravity,vwc,annually,soft,enough,spring,groundwater,communal standpipe,21
2,functional,Lottery Club,World vision,Pangani,Manyara,Simanjiro,Ngorika,True,VWC,True,gravity,vwc,per bucket,soft,enough,dam,surface,communal standpipe multiple,11
3,non functional,Unicef,UNICEF,Ruvuma / Southern Coast,Mtwara,Nanyumbu,Nanyumbu,True,VWC,True,submersible,vwc,never pay,soft,dry,machine dbh,groundwater,communal standpipe multiple,34
5,functional,Mkinga Distric Coun,DWE,Pangani,Tanga,Mkinga,Moa,True,VWC,True,submersible,vwc,per bucket,salty,enough,other,unknown,communal standpipe multiple,11
6,non functional,Dwsp,DWSP,Internal,Shinyanga,Shinyanga Rural,Samuye,True,VWC,True,handpump,vwc,never pay,soft,enough,machine dbh,groundwater,hand pump,23


In [128]:
#if we want to have a binary classification problem:
#df1['status_group'] = df1['status_group'].replace(to_replace =["functional needs repair", "non functional"],
                            #value ="needs repair")

###### **if we were to use get_dummies()...**

In [129]:
# # df_num = df[['amount_tsh', 'population', 'gps_height']]
# df_cat = df[['funder', 'scheme_management', 'extraction_type_group', 'management', 'water_quality', 'quantity', 'installer', 'basin']]

In [130]:
# df_dummies = pd.get_dummies(df_cat, drop_first=True)

# df_final = pd.concat([df_num, df_dummies], axis=1)
# df_final.shape

## OHE

In [131]:
# create y and X as the target and the features
y = df['status_group']
X = df.drop(columns='status_group')

In [132]:
cat_cols = X.select_dtypes(include=['object','bool']).columns.tolist()
cont_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols,cont_cols

(['funder',
  'installer',
  'basin',
  'region',
  'lga',
  'ward',
  'public_meeting',
  'scheme_management',
  'permit',
  'extraction_type_class',
  'management',
  'payment_type',
  'water_quality',
  'quantity',
  'source',
  'source_class',
  'waterpoint_type'],
 ['age'])

In [133]:
for col in cat_cols:
    X[col] = X[col].astype('category')

In [134]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state = 77, 
                                                    stratify = y, # in classification problems 
                                                                  # when you split the data 
                                                                  # you want to keep the ratio in the classes.
                                                    test_size = .2 # This is usually the ratio but it might change 
                                                                   # according to the problem at hand.
                                                   )

In [135]:
## create an encoder object. This will help us to convert
## categorical variables to new columns
ss = StandardScaler()
encoder = OneHotEncoder(handle_unknown= 'ignore',
                        #drop='first',
                        categories= 'auto')

## Create an columntransformer object.
## This will help us to merge transformed columns
## with the rest of the dataset.

ct = ColumnTransformer(transformers =[('ohe', encoder, cat_cols)], #('scaler', ss, cont_cols)],
                                    remainder= ss)
X_train_ohe = ct.fit_transform(X_train)
X_test_ohe = ct.transform(X_test)

In [136]:
#cats_ohe 

In [137]:
#cats_ohe = ct.get_feature_names()

In [138]:
X_train_ohe.shape, X_test_ohe.shape

((38920, 5135), (9731, 5135))

---

In [139]:
X_train_trans = pd.DataFrame(X_train_ohe.toarray(),columns=cats_ohe+cont_cols)

In [140]:
X_train_trans

,ohe__x0_0,ohe__x0_A/co Germany,ohe__x0_Aar,ohe__x0_Abas Ka,ohe__x0_Abasia,ohe__x0_Abc-ihushi Development Cent,ohe__x0_Abd,ohe__x0_Abood,ohe__x0_Abs,ohe__x0_Aco/germany,...,ohe__x15_surface,ohe__x15_unknown,ohe__x16_cattle trough,ohe__x16_communal standpipe,ohe__x16_communal standpipe multiple,ohe__x16_dam,ohe__x16_hand pump,ohe__x16_improved spring,ohe__x16_other,age
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.176609
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.468170
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-1.475567
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.176609
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.086820
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38915,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-1.281194
38916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.176609
38917,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.017764
38918,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.856917


In [11]:
# ss = StandardScaler()

# numeric_features = ['amount_tsh', 'population', 'gps_height']
# numeric_transformer = Pipeline(steps=[
#     ('scaler', StandardScaler())])

# categorical_features = ['funder', 'scheme_management', 'extraction_type_group', 'management', 'water_quality', 'quantity', 'installer', 'basin']
# categorical_transformer = Pipeline(steps=[
#     ("ordinal", OrdinalEncoder()),
#     ('onehot', OneHotEncoder(categories='auto'))])

# preprocessor = ColumnTransformer(
#     transformers=[
#         ('num', numeric_transformer, numeric_features),
#         ('cat', categorical_transformer, categorical_features)], remainder='drop')

# #let's use it now!

# X_train_tran = (preprocessor.fit_transform(X_train))
# X_train_tran

<38920x3113 sparse matrix of type '<class 'numpy.float64'>'
	with 428120 stored elements in Compressed Sparse Row format>

In [141]:
#X_train_tran = pd.DataFrame.sparse.from_spmatrix(X_train_tran)

In [149]:
y_train

55126    functional needs repair
45667                 functional
16512                 functional
52526                 functional
8183     functional needs repair
                  ...           
42077                 functional
52751             non functional
27505                 functional
38158                 functional
6041              non functional
Name: status_group, Length: 38920, dtype: object

logistic, random forest, xboost, (confusion, class report, metric (top 2 or 3) hypter par tune top models. 2 adequate. which one is more simplistic if approx same value.)

## Dummy Modeling

- Linear Regression
- Random Forest
- xboost


#### Logistic Regression

In [142]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='liblinear', penalty='l1')
log_reg.fit(X_train_trans, y_train)
log_reg.score(X_train_trans, y_train)

/Users/alphonsowoodbury/opt/miniconda3/envs/learn-env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8069630010277492

In [150]:
y_pred = log_reg.predict(X)

ValueError: could not convert string to float: 'Roman'

In [148]:
y_pred = log_reg.predict(X)
score = np.round(Log_reg.score(X,y), 3)
cm = confusion_matrix(y, y_pred)


plt.figure(figsize=(9,9))
sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Pastel1');
plt.ylabel('Actual label');
plt.xlabel('Predicted label');
all_sample_title = 'Accuracy Score: {0}'.format(score)
plt.title(all_sample_title, size = 15);
#plt.savefig('toy_Digits_ConfusionSeabornCodementor.png')
plt.show();

ValueError: could not convert string to float: 'Roman'

#### Random Forest

## Metrics

#### Confusion Matrix